# Part 3: Mapping the impact of wild fires
Now that we have the skills to work with geospatial data, we’re going to explore some of the things we can do with it!  
You already saw that our visual data consists of three different bands: red, green, and blue. However, the electromagnetic spectrum is much larger than only the visible light, and for our next exercise, we’ll also use the Near Infrared (NIR) light. By combining the red and near infrared light, we can compute an index to measure the ‘greenness’ (or chemically the chlorophyll) of plants. This index is called the Normalized Difference Vegetation Index, also known as the NDVI. And is often used as a measure of 'health' for plants.

Below is an example of typical NDVI values for different plants. As you can see, dead plants correspond to an NDVI value of zero or even negative. This is important to understand, with this, we can understand why the NDVI will decrease after a fire!

<div style="text-align:center">
  <img src="imgs/NDVI-values-by-crop-condition.jpeg" alt="NDVI values for various crop conditions" width="500" height="300">
</div>

[Image source](https://www.auravant.com/en/articles/precision-agriculture/vegetation-indices-and-their-interpretation-ndvi-gndvi-msavi2-ndre-and-ndwi/)

In this final exercise, we will combine our knowledge of vector and raster data to evaluate the effect of a forest fire on a region with different types of forest.   
We can immediately start with importing the libraries we know we need for the vector and raster analysis.

In [ ]:
# required libraries
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.plot import show

## Exploring the data
We can use our forest vector file we created in exercise 2. However, we will use a slightly different image for the raster data, which now contains the Near Infrared (NIR) band as well.

(You might notice that the values of the rasters are on a different scale. In the first exercise we used the *visual* image, which contains normalized values such that we can plot the RGB images easily. Now we'll work with the *surface reflectance* values instead, because we need the NIR band as well. For today we can just work with these values without a deeper physical understanding, but if you are interested, you can find a more elaborate explanation [here](https://support.planet.com/hc/en-us/articles/360000285067-What-is-Surface-Reflectance-).)

In [ ]:
# Load the raster data: before and after the forest fire
before_file_path = 'data/20230817_surface_reflectance_BGR-NIR.tif'
after_file_path = 'data/20230830_surface_reflectance_BGR-NIR.tif'

before_fire_img = rasterio.open(before_file_path)
after_fire_img = rasterio.open(after_file_path)

# check out the metadata
before_fire_img.meta

We can see that this time our image has `count` 4 instead of 3: this is our extra NIR band!  
Now let's take a look at the Red and NIR bands (the bands we need for the NDVI). 

In [ ]:
# extract the Red and NIR bands from the before and after image
fig, axs = plt.subplots(2, 2, figsize=(9, 7), sharex="all", sharey="all")

# Calculate the 95th percentile range for the images such that we can fairly compare them
vmin_red = np.percentile(np.concatenate([before_fire_img.read(1), after_fire_img.read(1)], axis=None), 2.5)
vmax_red = np.percentile(np.concatenate([before_fire_img.read(1), after_fire_img.read(1)], axis=None), 97.5)
vmin_nir = np.percentile(np.concatenate([before_fire_img.read(4), after_fire_img.read(4)], axis=None), 2.5)
vmax_nir = np.percentile(np.concatenate([before_fire_img.read(4), after_fire_img.read(4)], axis=None), 97.5)

# plot red and nir band before the wild fire
show(
    (before_fire_img, 1),
    ax=axs[0, 0],
    cmap="Reds",
    title="red band before",
    vmin=vmin_red,
    vmax=vmax_red,
)
show(
    (before_fire_img, 4),
    ax=axs[1, 0],
    cmap="viridis",
    title="nir band before",
    vmin=vmin_nir,
    vmax=vmax_nir,
)

# plot red and nir band after the wild fire
im_red = show(
    (after_fire_img, 1),
    ax=axs[0, 1],
    cmap="Reds",
    title="red band after",
    vmin=vmin_red,
    vmax=vmax_red,
)
im_nir = show(
    (after_fire_img, 4),
    ax=axs[1, 1],
    cmap="viridis",
    title="nir band after",
    vmin=vmin_nir,
    vmax=vmax_nir,
)

# Position and size the red colorbar on the right side of the second column
cax_red = fig.add_axes([axs[0, 1].get_position().x1 + 0.01, axs[0, 1].get_position().y0, 0.02, axs[0, 1].get_position().height])
cbar_red = plt.colorbar(im_red.get_images()[0], cax=cax_red)
cbar_red.set_label("Surface reflectance red band")
# same for the nir band
cax_nir = fig.add_axes([axs[1, 1].get_position().x1 + 0.01, axs[1, 1].get_position().y0, 0.02, axs[1, 1].get_position().height])
cbar_nir = plt.colorbar(im_nir.get_images()[0], cax=cax_nir)
cbar_nir.set_label("Surface reflectance NIR band")

# fig.tight_layout()
plt.subplots_adjust(wspace=0.1)
plt.show()
plt.close()

We can already see that the NIR reflectance is much lower after the wild fire. Looking at the NDVI formula again, 
$$
    NDVI = \frac{NIR - Red}{NIR + Red},
$$
it shows us that a high NIR signal corresponds to high NDVI. 

## Computing the NDVI
Now that we have our NIR and red band, we can compute the NDVI. To easily recompute the NDVI for the two images, let's define it as a function. 

In [ ]:
def compute_ndvi(
    img: rasterio.io.DatasetReader,
    red_band_nr: int = 1,
    nir_band_nr: int = 4,
) -> np.ndarray:
    """
    Compute the NDVI from a spectral reflectance image containing a red and NIR band.

    Args:
        img (np.ndarray): image with spectral bands
        red_band_nr (int, optional): number of red band. Defaults to 1.
        nir_band_nr (int, optional): number of NIR band. Defaults to 4.

    Returns:
        np.ndarray: NDVI image
    """
    # check if the img indeed has 4 bands
    if img.count < red_band_nr or img.count < nir_band_nr:
        raise ValueError(
            f"Assigned band number ({red_band_nr} or {nir_band_nr}) is too large. Image has {img.count} bands"
        )

    # get the bands from the image
    red_band = img.read(red_band_nr)
    nir_band = img.read(nir_band_nr)

    # compute ndvi and make sure not to divide by 0
    ndvi = np.divide((nir_band - red_band), (nir_band + red_band), where=(nir_band + red_band) != 0)

    return ndvi


# run the function to calculate the ndvi numpy arrays
before_fire_ndvi = compute_ndvi(before_fire_img)
after_fire_ndvi = compute_ndvi(after_fire_img)

So we got the NDVI now! However, as you might have noticed, the function returns the NDVI as a numpy array, which is not georeferenced. When we would plot the NDVI now, we can see that the axes represent the row and column indices from the numpy array rather than the geographical coordinates. 

In [ ]:
plt.imshow(before_fire_ndvi)
plt.show()
plt.close()

To restore the geolocation of the image, we can write the NDVI images to GeoTIFF files with the geotransform from the original images.

If you want to learn more about the projection and transform, check out the *Extra* section at the end of the raster notebook with exercise 1.

In [ ]:
# import the affine library for the reprojection
import affine

def save_raster(dataset: np.ndarray,
                dataset_path: str,
                cols: int,
                rows: int,
                projection: affine.Affine,
                transform: affine.Affine,
                ) -> None:
    """
    Save raster to tif file with its geotransform

    Args:
        dataset (np.ndarray): raster to store
        datasetPath (str): file path to output location
        cols (int): width of the output img (nr of pixels)
        rows (int): height of the output img (nr of pixels)
        projection (affine.Affine): crs of output image
        transform (affine.Affine): geotransform of output image
    """
    # Open a new GeoTIFF file for writing
    with rasterio.open(dataset_path, 'w',
                       driver='GTiff',
                       height=rows,
                       width=cols,
                       count=1,
                       dtype=np.float32,
                       crs=projection,
                       transform=transform) as dst:
        # Write the input dataset to the output file
        dst.write(dataset, 1)

In [ ]:
# define parameters
img_cols = before_fire_img.width
img_rows = before_fire_img.height
img_projection = before_fire_img.crs
img_transform = before_fire_img.transform

type(before_fire_ndvi)

# store our before and after ndvi to a tif file
save_raster(
    before_fire_ndvi,
    "data/ndvi_before.tif",
    cols=img_cols,
    rows=img_rows,
    projection=img_projection,
    transform=img_transform,
)
save_raster(
    after_fire_ndvi,
    "data/ndvi_after.tif",
    cols=img_cols,
    rows=img_rows,
    projection=img_projection,
    transform=img_transform,
)

If we plot our NDVI images again, we can see that the coordinates reappear on the axes.

Note that, in the plot below, we only look at the NDVI in the range from [0.33, 1.0]. This range, as you can see in the image at the top of the notebook, corresponds to moderately healthy vegetation and better. So by only plotting this part of the range, subtle differences are highlighted and low NDVI values (<0.33) are better visible.

In [ ]:
import matplotlib.ticker as ticker

# load our new images
ndvi_before = rasterio.open("data/ndvi_before.tif")
ndvi_after = rasterio.open("data/ndvi_after.tif")

# plot the NDVI before and after the wildfire next to each other
fig, axs = plt.subplots(1, 2, figsize=(12, 8), sharey=True)
ndvi_plot_before = show(
    (ndvi_before, 1), ax=axs[0], cmap="RdYlGn", title="NDVI before", vmin=0.33, vmax=1
)
ndvi_plot_after = show(
    (ndvi_after, 1), ax=axs[1], cmap="RdYlGn", title="NDVI after", vmin=0.33, vmax=1
)

# figure layout
cax = fig.add_axes([axs[1].get_position().x1 + 0.01, axs[1].get_position().y0, 0.02, axs[1].get_position().height])
cbar = plt.colorbar(ndvi_plot_after.get_images()[0], cax=cax)
cbar.set_label("NDVI")
plt.show()
plt.close()

If we want to know the difference between the before and after, we can simpy print the mean values. However, this value also contains the NDVI values for areas such as roads and cities. Therefore, it *underestimates* the effect of the fire on the natural areas. If we want to see the true effect on these natural areas, we need our vector data to extract the data in these locations.

## Combining vector and raster data
Let's start by loading our vector data of the forest areas and plotting it together with the raster images we created of the NDVI to visualize their relationship.

In [ ]:
# load the vector data
landcover = gpd.read_file("data/landcover_greece.geojson")
forest_landcover = gpd.read_file("data/forest_landcover.geojson")

# plot the boundaries over the raster to see which areas contain forest
fig, axs = plt.subplots(1, 2, figsize=(12, 8), sharey=True)
ndvi_plot_before = show((ndvi_before, 1), ax=axs[0], cmap="RdYlGn", vmin=0.33, vmax=1, title="NDVI before with forest boundaries")
ndvi_plot_after = show((ndvi_after, 1), ax=axs[1], cmap="RdYlGn", vmin=0.33, vmax=1, title="NDVI after with forest boundaries")
# plot the boundaries on both axes
[forest_landcover.boundary.plot(ax=ax, color="black") for ax in axs]

# plot layout
[ax.set_xlim(414000, 425000) for ax in axs]
cax = fig.add_axes([axs[1].get_position().x1 + 0.01, axs[1].get_position().y0, 0.02, axs[1].get_position().height])
cbar = plt.colorbar(ndvi_plot_after.get_images()[0], cax=cax)
cbar.set_label("NDVI")
plt.show()
plt.close()

As expected, before the wild fire occurred, the NDVI is a lot higher within forest areas than after the fire.

To see where the impact of the fire was the most severe, we can compute the difference in NDVI by subtracting the rasters from each other. 

In [ ]:
# calculate the difference before and after the fire
diff_ndvi = ndvi_after.read(1) - ndvi_before.read(1)

# now we can see that this again is in numpy array format, so we need to restore the geo-information by saving the result as raster
save_raster(
    diff_ndvi,
    "data/ndvi_difference.tif",
    cols=img_cols,
    rows=img_rows,
    projection=img_projection,
    transform=img_transform,
)

In [ ]:
# open the raster we just created
ndvi_diff_raster = rasterio.open("data/ndvi_difference.tif")

# plot the difference in NDVI
fig, ax = plt.subplots(figsize=(6, 5))
plot_ndvi_diff = show(ndvi_diff_raster, ax=ax, cmap="RdYlGn")

# plot layout
cax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height])
cbar = plt.colorbar(plot_ndvi_diff.get_images()[0], cax=cax)
cbar.set_label("Difference in NDVI")
plt.show()
plt.close()

That already clearly shows that there was a much greater impact on some parts of the area than on others!  
Now if we want to know which *area* (polygon) was the most damaged, we can use our vector file. By locating where the fire was the worst averaged over the area, we can determine which type of forest is most affected by the fire.

So let's start by averaging the NDVI for each (forest) polygon. 
We can start by adding the average NDVI values as an attribute to our geodataframe.

In [ ]:
# duplicate and visualize our original dataframe
forest_ndvi = forest_landcover.copy()
forest_landcover.head()

In [ ]:
from rasterio.mask import mask
from shapely.geometry import mapping, MultiPolygon


def compute_ndvi_within_multipolygon(
    raster_data: rasterio.io.DatasetReader,
    multipolygon: MultiPolygon,
) -> float:
    """Compute NDVI within a multipolygon

    Args:
        raster_data (rasterio.io.DatasetReader): rasterio raster data with NDVI values
        multipolygon (MultiPolygon): geometry of format multipolygon

    Returns:
        float: Average NDVI values
    """
    # create a list of the multipolygons
    # (mapping converts the multipolygons to a geojson like format)
    geojson_geometry = [mapping(multipolygon)]
    # The mask function from rasterio is used to extract the values from the raster_data within the multipolygon
    # crop=True parameter ensures that the extracted data aligns with the boundaries of the multipolygon
    masked_data, _ = mask(raster_data, geojson_geometry, crop=True)
    # average the values within the polygons
    average_ndvi = np.nanmean(masked_data)

    return average_ndvi

In [ ]:
# apply function to geodataframe
forest_ndvi["Average_ndvi_before"] = forest_ndvi.apply(
    lambda row: compute_ndvi_within_multipolygon(ndvi_before, row["geometry"]), axis=1
)
forest_ndvi["Average_ndvi_after"] = forest_ndvi.apply(
    lambda row: compute_ndvi_within_multipolygon(ndvi_after, row["geometry"]), axis=1
)
forest_ndvi.head()

## Exercise
Can you find out where (for which forest polygon) the impact of the fire was the most severe?

In [ ]:
## solutions
# calculate the difference between the NDVI before and after
wildfire_impact = forest_ndvi.copy()
# [YOUR ANSWER HERE]

# locate the largest difference
row_with_highest_ndvi_difference = # [YOUR ANSWER HERE]

# (Hint: use the idxmax() function)

## Visualizing the area
Now that we found the area which was damaged most, we can visualize the area with our raster and vector.

In [ ]:
# filter the dataframe by the row with the largest impact
geom_largest_diff = gpd.GeoDataFrame([row_with_highest_ndvi_difference], geometry="geometry")

# plot the boundaries over the raster to see which areas contain forest
fig, axs = plt.subplots(figsize=(10, 7))
ndvi_diff_plot = show((ndvi_diff_raster, 1), ax=axs, cmap="RdYlGn", title="Most Damaged Area")
gpd.plotting.plot_dataframe(geom_largest_diff, ax=axs, facecolor="none", edgecolor="black", lw=4)

# plot layout
cax = fig.add_axes([ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height])
cbar = plt.colorbar(ndvi_diff_plot.get_images()[0], cax=cax)
cbar.set_label("Difference in NDVI")
plt.show()
plt.close()

Well done, you made it to the end! Thank you for your participation and don't forget to close your datasets ;)

In [ ]:
# close all datasets
ndvi_diff_raster.close()
before_fire_img.close()
after_fire_img.close()
ndvi_before.close()
ndvi_after.close()